In [37]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
import random
from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import mutual_info_classif
import warnings

In [38]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [39]:
train = train.sample(460)

In [40]:
X = train.drop(columns=['SalePrice', 'Id']).copy()
y = train['SalePrice']

In [47]:
X.isnull().sum().sort_values(ascending=False)

Fence            380
MasVnrType       283
FireplaceQu      211
LotFrontage       90
GarageType        22
                ... 
TotalBsmtSF        0
Heating            0
MSZoning           0
CentralAir         0
SaleCondition      0
Length: 76, dtype: int64

In [42]:
Missing = X.isna().mean()*100 # percentage of missing columns
High_missing_columns = Missing[Missing>85]
High_missing_columns.reset_index() 
High_missing_columns.sort_values(ascending=False,inplace =True)
High_missing_columns

PoolQC         99.347826
MiscFeature    97.173913
Alley          93.913043
dtype: float64

In [43]:
def droping_columns(df):
    Missing = df.isna().mean()*100
    colums_to_drop = df.columns[Missing>85]
    df.drop(columns = colums_to_drop, inplace=True)
    return df

get_droping_columns = FunctionTransformer(droping_columns)

In [45]:
X = droping_columns(X)

In [ ]:
X

In [21]:
X_1 = train.drop(columns=['SalePrice']).copy()
y = train['SalePrice']

In [22]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold,KFold,RepeatedStratifiedKFold,RepeatedKFold
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.feature_selection import SelectFromModel, RFECV, SelectKBest, chi2, f_classif,f_regression, SelectFromModel, RFE, mutual_info_classif, SelectPercentile
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, r2_score, roc_auc_score
import optuna

In [23]:
# splitting the training dataframe into x(features) and y (target)
X_train, X_valid, y_train, y_valid = train_test_split(X_1,y, train_size = 0.8, random_state=42)

In [24]:
# The class of the function that will perform the winsorization on the data based on the parameters learnt during the fit process 
from scipy.stats.mstats import winsorize

class Custom_winsorize(): 

    # The constructor of the class that will initialize the properties of the object 
    def __init__(self): 
        
        # Numeric features in the data 
        self.numeric_cols = []
        
        # Store the lower and the upper whisker values for each feature in the data 
        # A tuple for each column would be stored in a dictionary
        self.whiskers = dict()
        
    # This function would help in order to identify the lower and the upper bound of the whiskers 
    def outlier_bound(self, X, column):
    
        # Getting the 25th percentile of the column 
        q1= np.percentile(X[column], 25)

        # Getting the 75th percentile of the column 
        q3= np.percentile(X[column], 75)

        # Computing the IQR for the data 
        iqr=q3-q1

        # Getting the upper whisker of the data
        upper_whisker = q3 + 1.5*(iqr)

        # Getting the lowe whisker of the data
        lower_whisker = q1 - 1.5*(iqr)

    
        # Return the upper and the lowe whisker of the data 
        return (round(lower_whisker, 2), round(upper_whisker, 2))

    # This function would help in order to identify the whiskers value for each column in the data 
    def fit(self, X, y = None ): 
        
        # Convert the data to a dataframe to make sure that we are working on a dataframe
        X = pd.DataFrame(X)
        
        # Identify the numeric features in the data
        self.numeric_cols = [col for col in X.columns if X[col].dtype != 'object']
        
        # Get the whiskers bound for each feature in the data 
        for column in self.numeric_cols:
        
            # Get the upper and the lower value of the whiskers 
            self.whiskers[column] = self.outlier_bound(X, column)
        

        # Return the object itself 
        return self
    
    # Transform the data based on the found whiskers values 
    def transform(self, X, y = None):
    
        # Take a copy of the initial data 
        Temp_X = pd.DataFrame(X).copy()
        
        # Loop over the numeric features that we have in the data 
        for column in self.numeric_cols:
            
            # Take the lower and the upper bound of the whiskers for this column 
            lower, upper = self.whiskers[column]
            
            # Get the percentage of the samples that should be winsorized in the lower bound
            limit_lower_percent = Temp_X[(Temp_X[column] < lower)].shape[0] / Temp_X.shape[0]
            
            # Get the percentage of the sampels that should be winsorized in the upper bound
            limit_upper_percent = Temp_X[(Temp_X[column] > upper)].shape[0] / Temp_X.shape[0]
            
            # Applying the winsorization given the parameters for this column
            Temp_X[column] = winsorize(Temp_X[column], limits=[limit_lower_percent, limit_upper_percent])
            
        # Return the winsorized version of the data
        return Temp_X

customwinsorize = Custom_winsorize()

In [25]:
# Aggregate numerical and categorical
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson', standardize=True)),
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])
# Preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols),
    ])

In [26]:
# Aggregate numerical and categorical
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

# Numeric Columns - imputing missing values
imp_median = SimpleImputer(strategy='median', add_indicator=True)
imp_knn = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
minmaxscaler = MinMaxScaler()
scaler = StandardScaler()

# Categorical Columns - imputing missing values
imp_constant = SimpleImputer(strategy='constant', fill_value='missing')
ohe = OneHotEncoder(handle_unknown='ignore')
ori = OrdinalEncoder()
selector= SelectPercentile(chi2, percentile=50)

# power = PowerTransformer(method='box-cox', standardize=True) #only works with strictly positive values
power = PowerTransformer(method='yeo-johnson', standardize=True) #default-works with positive and negative values

preprocessor_1 = make_column_transformer(
                                        (make_pipeline(imp_median, customwinsorize, scaler, power), num_cols),
                                        (make_pipeline(imp_constant, ohe), cat_cols)
                                      )

In [34]:
preprocessor_2 = make_column_transformer(
                                        (make_pipeline(imp_median,customwinsorize, minmaxscaler), num_cols),
                                        (make_pipeline(imp_constant, ohe), cat_cols) 
                                        )

In [27]:
# Feature selection
selection = SelectFromModel(Lasso(alpha=0.3, random_state=42))
selection_1 = RFE(DecisionTreeRegressor(random_state=42), n_features_to_select=100)
selection_2 = SelectKBest(f_regression)

<a href="#toc" role="button" aria-pressed="true" >🔝Back to Table of Contents🔝</a>

<a id="V"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700">V <span style='color:#CDA63A'>|</span>MODEL ENGINEERING 🤖</div>

Root-Mean-Squared-Error (RMSE)

 ### <a id="V.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.1 RandomForestRegressor</p>

In [35]:
pipe_1 = make_pipeline(preprocessor_2, selection,  RandomForestRegressor(n_estimators=100, random_state=42, oob_score=True))

In [36]:
pipe_1.fit(X_train, y_train)

D:\Programy\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.221e+10, tolerance: 2.490e+08
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('custom_winsorize',
                                                                   <__main__.Custom_winsorize object at 0x000001E35B1DE410>),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000...
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E35F7DCCD0>)])),
                ('selectfrommodel',
                 SelectFromModel(estimator=Lasso(alpha=0.3, random_state=42))),
                ('randomforestregressor',
                 RandomForestRegressor(oob_score=True, random_state=42))])

 ### <a id="V.1.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.1.1 RandomForestRegressor - Evaluation</p>

In [179]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = cross_val_score(pipe_1, X, y, cv=kf, scoring='r2')
print('R2_score: %.2f (%.2f)'% (np.mean(n_scores), np.std(n_scores)))

R2_score: 0.86 (0.03)


In [164]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = -cross_val_score(pipe_1, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print("MAE scores:\n", n_scores)
# print('Accuracy: %.2f (%.2f)'% (np.mean(n_scores)))
print("Average MAE score (across experiments):")
print(n_scores.mean())

MAE scores:
 [27371.14895405 25925.99319039 39318.98714358 22685.34030352
 26209.06187973]
Average MAE score (across experiments):
28302.106294251782


In [78]:
predictions_1 = pipe_1.predict(X_valid)

array([210359.04, 162132.97, 127650.73,  92252.5 , 125559.64, 158562.94,
       228758.54, 169544.32, 146648.2 , 163512.29, 201752.32, 119545.04,
       248384.  , 277068.72, 171539.55,  77932.56, 184297.32, 213091.93,
       134845.62, 167777.72, 184893.27, 142692.2 , 222371.74, 178230.96,
       108241.08, 304172.33, 171756.  , 183356.66, 124992.53,  91679.5 ,
       127812.  , 176248.52, 260442.28, 135332.5 , 162729.82, 234891.8 ,
       125310.87, 282255.04, 136492.15, 133636.59, 110128.78, 131433.46,
       125555.83, 196741.22, 344438.77, 140052.54, 156371.22, 132172.39,
       256634.5 , 131555.56, 239189.54, 136240.  , 156711.78, 126317.34,
       382207.43, 131191.45, 245332.87, 439085.2 , 264844.99,  92519.04,
       251417.21, 159611.5 , 154253.28, 119973.7 , 125915.42, 197545.78,
       191107.  , 115219.44, 123796.02, 142758.86, 279664.79, 120198.2 ,
       122882.6 , 195986.57, 280569.22, 154974.5 , 190969.86, 138673.56,
       170950.  , 244493.57, 139360.44, 150387.81, 

### <a id="V.1.2"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.1.2 RandomForestRegressor tuning - Optuna</p>

### <a id="V.1.3"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.1.3 RandomForestRegressor tuning - Optuna - evaluation</p>

 ### <a id="V.2"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.2 XGBRegressor</p>

In [169]:
pipe_2 = make_pipeline(preprocessor, selection,  XGBRegressor(random_state=42))

In [170]:
pipe_2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('powertransformer',
                                                                   PowerTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D0107990>),
                                                 ('pipeline-2',
                                                  Pip...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

 ### <a id="V.2.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.2.1 XGBRegressor - Evaluation</p>

In [180]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = cross_val_score(pipe_2, X, y, cv=kf, scoring='r2')
print('R2_score: %.2f (%.2f)'% (np.mean(n_scores), np.std(n_scores)))

R2_score: 0.83 (0.04)


In [181]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = -cross_val_score(pipe_2, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print("MAE scores:\n", n_scores)
# print('Accuracy: %.2f (%.2f)'% (np.mean(n_scores)))
print("Average MAE score (across experiments):")
print(n_scores.mean())

MAE scores:
 [27714.25353971 33753.35030169 37408.04638855 26335.83918954
 28357.69851499]
Average MAE score (across experiments):
30713.837586894522


 ### <a id="V.3"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.3 LGBMRegressor</p>

In [183]:
from lightgbm import LGBMRegressor

In [184]:
pipe_3 = make_pipeline(preprocessor, selection,  LGBMRegressor(random_state=42))

In [185]:
pipe_3.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1727
[LightGBM] [Info] Number of data points in the train set: 460, number of used features: 73
[LightGBM] [Info] Start training from score 183011.867391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('powertransformer',
                                                                   PowerTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D0107990>),
                                                 ('pipeline-2',
                                                  Pip...
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D017B6D0>)])),
                ('selectfrommodel',
                 SelectFromModel(estimator=Lasso(alpha=0.3, random_state=42))),
                ('lgbmregressor', LGBMRegressor(random_state=42))])

 ### <a id="V.3.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.3.1 LGBMRegressor - Evaluation</p>

In [186]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = cross_val_score(pipe_3, X, y, cv=kf, scoring='r2')
print('R2_score: %.2f (%.2f)'% (np.mean(n_scores), np.std(n_scores)))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1435
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 66
[LightGBM] [Info] Start training from score 183117.043478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000360 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1439
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 71
[LightGBM] [Info] Start training from score 181125.913043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1455
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 75
[LightGBM] [Info] Start training from score 183464.763587
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [187]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = -cross_val_score(pipe_3, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print("MAE scores:\n", n_scores)
# print('Accuracy: %.2f (%.2f)'% (np.mean(n_scores)))
print("Average MAE score (across experiments):")
print(n_scores.mean())

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000328 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1435
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 66
[LightGBM] [Info] Start training from score 183117.043478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1439
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 71
[LightGBM] [Info] Start training from score 181125.913043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1455
[LightGBM] [Info] Number of data points in the train set: 368, number of used features: 75
[LightGBM] [Info] Start training from score 183464.763587
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

 ### <a id="V.4"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.4 CatBoostRegressor</p>

In [192]:
from catboost import CatBoostRegressor

In [193]:
pipe_4 = make_pipeline(preprocessor, selection,  CatBoostRegressor(random_state=42))

In [194]:
pipe_4.fit(X,y)

Learning rate set to 0.036216
0:	learn: 75605.3421436	total: 140ms	remaining: 2m 19s
1:	learn: 74111.1271462	total: 142ms	remaining: 1m 10s
2:	learn: 72522.9530290	total: 143ms	remaining: 47.6s
3:	learn: 71074.3365396	total: 145ms	remaining: 36.1s
4:	learn: 69834.8518878	total: 147ms	remaining: 29.2s
5:	learn: 68550.4868111	total: 148ms	remaining: 24.6s
6:	learn: 67373.4551917	total: 150ms	remaining: 21.3s
7:	learn: 66207.0943169	total: 152ms	remaining: 18.8s
8:	learn: 64877.3285677	total: 154ms	remaining: 16.9s
9:	learn: 63687.2072771	total: 155ms	remaining: 15.4s
10:	learn: 62596.4202175	total: 157ms	remaining: 14.1s
11:	learn: 61498.5474559	total: 159ms	remaining: 13.1s
12:	learn: 60284.5699970	total: 160ms	remaining: 12.2s
13:	learn: 59170.3410816	total: 162ms	remaining: 11.4s
14:	learn: 58159.0333670	total: 164ms	remaining: 10.8s
15:	learn: 57103.8303181	total: 166ms	remaining: 10.2s
16:	learn: 56213.0090362	total: 167ms	remaining: 9.67s
17:	learn: 55196.7079063	total: 169ms	remai

165:	learn: 17180.7636454	total: 399ms	remaining: 2s
166:	learn: 17134.2132619	total: 401ms	remaining: 2s
167:	learn: 17056.4470850	total: 402ms	remaining: 1.99s
168:	learn: 17000.7318051	total: 404ms	remaining: 1.99s
169:	learn: 16929.3659115	total: 406ms	remaining: 1.98s
170:	learn: 16842.3626521	total: 407ms	remaining: 1.97s
171:	learn: 16794.8449645	total: 409ms	remaining: 1.97s
172:	learn: 16755.9043811	total: 411ms	remaining: 1.96s
173:	learn: 16744.3985391	total: 413ms	remaining: 1.96s
174:	learn: 16650.7161761	total: 414ms	remaining: 1.95s
175:	learn: 16594.1923198	total: 416ms	remaining: 1.95s
176:	learn: 16559.8103030	total: 418ms	remaining: 1.94s
177:	learn: 16520.5289636	total: 420ms	remaining: 1.94s
178:	learn: 16456.8893982	total: 421ms	remaining: 1.93s
179:	learn: 16421.5475584	total: 423ms	remaining: 1.93s
180:	learn: 16350.5828724	total: 425ms	remaining: 1.92s
181:	learn: 16336.3897041	total: 426ms	remaining: 1.92s
182:	learn: 16283.6983592	total: 428ms	remaining: 1.91

412:	learn: 8169.4215641	total: 793ms	remaining: 1.13s
413:	learn: 8160.4984351	total: 794ms	remaining: 1.12s
414:	learn: 8139.6517857	total: 796ms	remaining: 1.12s
415:	learn: 8133.3325320	total: 797ms	remaining: 1.12s
416:	learn: 8106.9908702	total: 799ms	remaining: 1.12s
417:	learn: 8102.5129821	total: 801ms	remaining: 1.11s
418:	learn: 8073.9429286	total: 803ms	remaining: 1.11s
419:	learn: 8040.7444912	total: 809ms	remaining: 1.12s
420:	learn: 8036.3045464	total: 811ms	remaining: 1.11s
421:	learn: 8031.6841210	total: 813ms	remaining: 1.11s
422:	learn: 8006.2902613	total: 815ms	remaining: 1.11s
423:	learn: 7984.9342860	total: 817ms	remaining: 1.11s
424:	learn: 7972.4621128	total: 818ms	remaining: 1.11s
425:	learn: 7942.1973467	total: 820ms	remaining: 1.1s
426:	learn: 7922.4585131	total: 822ms	remaining: 1.1s
427:	learn: 7890.7471463	total: 823ms	remaining: 1.1s
428:	learn: 7858.0364379	total: 825ms	remaining: 1.1s
429:	learn: 7850.1707566	total: 826ms	remaining: 1.09s
430:	learn: 78

648:	learn: 4916.8399100	total: 1.18s	remaining: 638ms
649:	learn: 4906.7572347	total: 1.18s	remaining: 636ms
650:	learn: 4898.0335976	total: 1.18s	remaining: 634ms
651:	learn: 4882.6181192	total: 1.18s	remaining: 632ms
652:	learn: 4862.0230621	total: 1.19s	remaining: 630ms
653:	learn: 4844.1355052	total: 1.19s	remaining: 628ms
654:	learn: 4837.6273238	total: 1.19s	remaining: 626ms
655:	learn: 4836.1304836	total: 1.19s	remaining: 625ms
656:	learn: 4832.7968714	total: 1.19s	remaining: 623ms
657:	learn: 4823.9498697	total: 1.19s	remaining: 621ms
658:	learn: 4812.8263110	total: 1.2s	remaining: 619ms
659:	learn: 4811.6195998	total: 1.2s	remaining: 617ms
660:	learn: 4796.5553461	total: 1.2s	remaining: 615ms
661:	learn: 4786.9550972	total: 1.2s	remaining: 613ms
662:	learn: 4784.8889265	total: 1.2s	remaining: 612ms
663:	learn: 4783.4818895	total: 1.2s	remaining: 610ms
664:	learn: 4767.5702761	total: 1.21s	remaining: 608ms
665:	learn: 4758.8340042	total: 1.21s	remaining: 606ms
666:	learn: 4748

896:	learn: 2942.1509461	total: 1.58s	remaining: 181ms
897:	learn: 2935.5165300	total: 1.58s	remaining: 180ms
898:	learn: 2930.3570809	total: 1.58s	remaining: 178ms
899:	learn: 2922.4351102	total: 1.58s	remaining: 176ms
900:	learn: 2912.2967873	total: 1.59s	remaining: 174ms
901:	learn: 2904.3768752	total: 1.59s	remaining: 173ms
902:	learn: 2895.0360702	total: 1.59s	remaining: 171ms
903:	learn: 2887.8620443	total: 1.59s	remaining: 169ms
904:	learn: 2881.7609823	total: 1.59s	remaining: 167ms
905:	learn: 2872.4570996	total: 1.59s	remaining: 166ms
906:	learn: 2865.9612933	total: 1.6s	remaining: 164ms
907:	learn: 2858.0178064	total: 1.6s	remaining: 162ms
908:	learn: 2855.6192192	total: 1.6s	remaining: 160ms
909:	learn: 2846.9435635	total: 1.6s	remaining: 158ms
910:	learn: 2839.4648301	total: 1.6s	remaining: 157ms
911:	learn: 2838.1433581	total: 1.6s	remaining: 155ms
912:	learn: 2831.7548757	total: 1.61s	remaining: 153ms
913:	learn: 2825.0581829	total: 1.61s	remaining: 151ms
914:	learn: 2817

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('powertransformer',
                                                                   PowerTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D0107990>),
                                                 ('pipeline-2',
                                                  Pip...
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D017B6D0>)])),
                ('selectfrommodel',
                 SelectFromModel(estimator=Lasso(alpha=0.3, random_state=42))),
                ('catboostregressor',
                 <catboost.core.CatBoostRegressor object at 0x000001A1D51DD290>)])

 ### <a id="V.4.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.4.1 CatBoostRegressor - evaluation</p>

In [195]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = cross_val_score(pipe_4, X, y, cv=kf, scoring='r2')
print('R2_score: %.2f (%.2f)'% (np.mean(n_scores), np.std(n_scores)))

Learning rate set to 0.034961
0:	learn: 73439.4954875	total: 1.95ms	remaining: 1.94s
1:	learn: 72020.8933542	total: 3.84ms	remaining: 1.91s
2:	learn: 70680.2225818	total: 5.45ms	remaining: 1.81s
3:	learn: 69495.4921986	total: 7.08ms	remaining: 1.76s
4:	learn: 68125.5598834	total: 8.84ms	remaining: 1.76s
5:	learn: 66825.8088950	total: 10.6ms	remaining: 1.76s
6:	learn: 65789.6616592	total: 12.2ms	remaining: 1.73s
7:	learn: 64554.8482952	total: 13.8ms	remaining: 1.71s
8:	learn: 63482.2481349	total: 15.5ms	remaining: 1.71s
9:	learn: 62348.8341162	total: 17.2ms	remaining: 1.7s
10:	learn: 61358.3298858	total: 18.9ms	remaining: 1.7s
11:	learn: 60439.6405262	total: 20.7ms	remaining: 1.7s
12:	learn: 59495.4976191	total: 22.4ms	remaining: 1.7s
13:	learn: 58404.7101016	total: 24.1ms	remaining: 1.7s
14:	learn: 57274.4129299	total: 25.8ms	remaining: 1.69s
15:	learn: 56337.1718016	total: 27.5ms	remaining: 1.69s
16:	learn: 55392.3717463	total: 29.3ms	remaining: 1.7s
17:	learn: 54608.6825708	total: 31

254:	learn: 14192.3305726	total: 390ms	remaining: 1.14s
255:	learn: 14138.6476434	total: 392ms	remaining: 1.14s
256:	learn: 14074.1333294	total: 393ms	remaining: 1.14s
257:	learn: 14032.6942081	total: 395ms	remaining: 1.14s
258:	learn: 13992.3829171	total: 396ms	remaining: 1.13s
259:	learn: 13921.2528406	total: 398ms	remaining: 1.13s
260:	learn: 13902.5705041	total: 399ms	remaining: 1.13s
261:	learn: 13884.3146132	total: 401ms	remaining: 1.13s
262:	learn: 13837.7209404	total: 403ms	remaining: 1.13s
263:	learn: 13820.6969203	total: 404ms	remaining: 1.13s
264:	learn: 13785.5033766	total: 406ms	remaining: 1.13s
265:	learn: 13738.7711651	total: 407ms	remaining: 1.12s
266:	learn: 13708.3645677	total: 409ms	remaining: 1.12s
267:	learn: 13676.2889275	total: 410ms	remaining: 1.12s
268:	learn: 13634.1145713	total: 412ms	remaining: 1.12s
269:	learn: 13555.1768203	total: 413ms	remaining: 1.12s
270:	learn: 13499.4387948	total: 415ms	remaining: 1.12s
271:	learn: 13477.5025066	total: 416ms	remaining

524:	learn: 6395.2201481	total: 799ms	remaining: 723ms
525:	learn: 6391.7464129	total: 801ms	remaining: 722ms
526:	learn: 6355.5026365	total: 803ms	remaining: 720ms
527:	learn: 6347.0959876	total: 804ms	remaining: 719ms
528:	learn: 6327.7617062	total: 806ms	remaining: 717ms
529:	learn: 6304.6248956	total: 807ms	remaining: 716ms
530:	learn: 6293.8965024	total: 809ms	remaining: 715ms
531:	learn: 6269.3248972	total: 811ms	remaining: 713ms
532:	learn: 6259.2506367	total: 812ms	remaining: 712ms
533:	learn: 6246.3178872	total: 814ms	remaining: 710ms
534:	learn: 6219.4192850	total: 816ms	remaining: 709ms
535:	learn: 6191.3190379	total: 817ms	remaining: 708ms
536:	learn: 6172.2336992	total: 819ms	remaining: 706ms
537:	learn: 6167.1258404	total: 821ms	remaining: 705ms
538:	learn: 6141.2757609	total: 822ms	remaining: 703ms
539:	learn: 6129.7403042	total: 824ms	remaining: 702ms
540:	learn: 6102.3646804	total: 826ms	remaining: 701ms
541:	learn: 6088.7003238	total: 827ms	remaining: 699ms
542:	learn

781:	learn: 3413.5296678	total: 1.2s	remaining: 334ms
782:	learn: 3411.2151628	total: 1.2s	remaining: 332ms
783:	learn: 3399.0976387	total: 1.2s	remaining: 331ms
784:	learn: 3384.3031521	total: 1.2s	remaining: 329ms
785:	learn: 3377.8271494	total: 1.2s	remaining: 328ms
786:	learn: 3374.2257264	total: 1.21s	remaining: 326ms
787:	learn: 3363.3246785	total: 1.21s	remaining: 325ms
788:	learn: 3348.0671973	total: 1.21s	remaining: 323ms
789:	learn: 3334.3265311	total: 1.21s	remaining: 322ms
790:	learn: 3323.8101495	total: 1.21s	remaining: 320ms
791:	learn: 3314.1011741	total: 1.21s	remaining: 319ms
792:	learn: 3302.5439855	total: 1.22s	remaining: 317ms
793:	learn: 3286.6964883	total: 1.22s	remaining: 316ms
794:	learn: 3274.2060534	total: 1.22s	remaining: 314ms
795:	learn: 3263.9698122	total: 1.22s	remaining: 313ms
796:	learn: 3259.5269764	total: 1.22s	remaining: 311ms
797:	learn: 3254.1214473	total: 1.22s	remaining: 310ms
798:	learn: 3244.9181455	total: 1.23s	remaining: 308ms
799:	learn: 323

Learning rate set to 0.034961
0:	learn: 76928.9966725	total: 1.75ms	remaining: 1.75s
1:	learn: 75452.9849085	total: 3.37ms	remaining: 1.68s
2:	learn: 73911.3089117	total: 4.99ms	remaining: 1.66s
3:	learn: 72576.6738557	total: 6.57ms	remaining: 1.63s
4:	learn: 71242.2451990	total: 8.16ms	remaining: 1.62s
5:	learn: 70019.3028925	total: 9.69ms	remaining: 1.6s
6:	learn: 68765.3876167	total: 11.1ms	remaining: 1.58s
7:	learn: 67461.2389110	total: 12.6ms	remaining: 1.56s
8:	learn: 66452.3035244	total: 14.1ms	remaining: 1.56s
9:	learn: 65390.8605182	total: 15.6ms	remaining: 1.54s
10:	learn: 64396.9886740	total: 17.1ms	remaining: 1.54s
11:	learn: 63370.2740440	total: 18.6ms	remaining: 1.53s
12:	learn: 62265.5349098	total: 20.1ms	remaining: 1.53s
13:	learn: 61350.1652329	total: 21.6ms	remaining: 1.52s
14:	learn: 60234.2327397	total: 23ms	remaining: 1.51s
15:	learn: 59218.3331132	total: 24.5ms	remaining: 1.51s
16:	learn: 58245.3568329	total: 26ms	remaining: 1.5s
17:	learn: 57280.5141966	total: 27

...

766:	learn: 2999.9108025	total: 1.18s	remaining: 358ms
767:	learn: 2992.9932666	total: 1.18s	remaining: 357ms
768:	learn: 2984.5090645	total: 1.18s	remaining: 355ms
769:	learn: 2974.3027940	total: 1.18s	remaining: 354ms
770:	learn: 2973.7063652	total: 1.19s	remaining: 352ms
771:	learn: 2967.4871186	total: 1.19s	remaining: 350ms
772:	learn: 2958.2765137	total: 1.19s	remaining: 349ms
773:	learn: 2948.9827961	total: 1.19s	remaining: 347ms
774:	learn: 2943.8057212	total: 1.19s	remaining: 346ms
775:	learn: 2937.6622363	total: 1.19s	remaining: 344ms
776:	learn: 2933.2596774	total: 1.19s	remaining: 343ms
777:	learn: 2924.3386068	total: 1.2s	remaining: 341ms
778:	learn: 2912.7875090	total: 1.2s	remaining: 340ms
779:	learn: 2908.0697754	total: 1.2s	remaining: 338ms
780:	learn: 2907.3852212	total: 1.2s	remaining: 337ms
781:	learn: 2904.4884885	total: 1.2s	remaining: 335ms
782:	learn: 2903.6635551	total: 1.2s	remaining: 334ms
783:	learn: 2893.4886480	total: 1.21s	remaining: 332ms
784:	learn: 2880

Learning rate set to 0.034961
0:	learn: 76731.3955966	total: 1.88ms	remaining: 1.88s
1:	learn: 75256.6323407	total: 3.51ms	remaining: 1.75s
2:	learn: 74113.6122058	total: 5.23ms	remaining: 1.74s
3:	learn: 72882.3655416	total: 6.91ms	remaining: 1.72s
4:	learn: 71438.6667487	total: 8.45ms	remaining: 1.68s
5:	learn: 70227.1307089	total: 9.96ms	remaining: 1.65s
6:	learn: 69078.0583396	total: 11.6ms	remaining: 1.65s
7:	learn: 67795.0959557	total: 13.3ms	remaining: 1.65s
8:	learn: 66598.6485833	total: 15ms	remaining: 1.65s
9:	learn: 65461.9938538	total: 16.5ms	remaining: 1.63s
10:	learn: 64169.2888446	total: 18.1ms	remaining: 1.63s
11:	learn: 63016.3719314	total: 19.8ms	remaining: 1.63s
12:	learn: 61980.9879210	total: 21.6ms	remaining: 1.64s
13:	learn: 60960.5615893	total: 23.2ms	remaining: 1.64s
14:	learn: 59826.8842013	total: 25ms	remaining: 1.64s
15:	learn: 58746.5189223	total: 26.5ms	remaining: 1.63s
16:	learn: 57745.8080528	total: 28.1ms	remaining: 1.62s
17:	learn: 56671.2900779	total: 

247:	learn: 13565.5015843	total: 393ms	remaining: 1.19s
248:	learn: 13535.7802629	total: 395ms	remaining: 1.19s
249:	learn: 13477.4482746	total: 396ms	remaining: 1.19s
250:	learn: 13448.0675669	total: 398ms	remaining: 1.19s
251:	learn: 13419.8814231	total: 399ms	remaining: 1.18s
252:	learn: 13348.3251537	total: 401ms	remaining: 1.18s
253:	learn: 13273.1246068	total: 402ms	remaining: 1.18s
254:	learn: 13207.2073575	total: 404ms	remaining: 1.18s
255:	learn: 13186.4790642	total: 405ms	remaining: 1.18s
256:	learn: 13177.2024899	total: 407ms	remaining: 1.18s
257:	learn: 13115.2052510	total: 408ms	remaining: 1.17s
258:	learn: 13098.5677394	total: 410ms	remaining: 1.17s
259:	learn: 13066.1447314	total: 411ms	remaining: 1.17s
260:	learn: 13055.0274959	total: 413ms	remaining: 1.17s
261:	learn: 12991.9805182	total: 414ms	remaining: 1.17s
262:	learn: 12932.0874430	total: 416ms	remaining: 1.17s
263:	learn: 12880.8070760	total: 417ms	remaining: 1.16s
264:	learn: 12848.4489279	total: 419ms	remaining

507:	learn: 6466.5407708	total: 782ms	remaining: 758ms
508:	learn: 6446.1818686	total: 784ms	remaining: 756ms
509:	learn: 6421.1670920	total: 786ms	remaining: 755ms
510:	learn: 6419.1844162	total: 787ms	remaining: 753ms
511:	learn: 6405.7474551	total: 789ms	remaining: 752ms
512:	learn: 6392.9347266	total: 790ms	remaining: 750ms
513:	learn: 6367.1456992	total: 792ms	remaining: 749ms
514:	learn: 6347.0172148	total: 794ms	remaining: 747ms
515:	learn: 6344.2314518	total: 795ms	remaining: 746ms
516:	learn: 6327.5050041	total: 797ms	remaining: 744ms
517:	learn: 6314.0122543	total: 798ms	remaining: 743ms
518:	learn: 6281.5904175	total: 800ms	remaining: 741ms
519:	learn: 6251.0736134	total: 801ms	remaining: 740ms
520:	learn: 6224.4479393	total: 803ms	remaining: 738ms
521:	learn: 6204.8604840	total: 804ms	remaining: 736ms
522:	learn: 6191.2703016	total: 806ms	remaining: 735ms
523:	learn: 6169.7432795	total: 807ms	remaining: 733ms
524:	learn: 6159.6469498	total: 809ms	remaining: 732ms
525:	learn

763:	learn: 3490.9696482	total: 1.17s	remaining: 362ms
764:	learn: 3482.0673178	total: 1.17s	remaining: 361ms
765:	learn: 3475.3184189	total: 1.18s	remaining: 359ms
766:	learn: 3463.1659940	total: 1.18s	remaining: 358ms
767:	learn: 3455.9224899	total: 1.18s	remaining: 356ms
768:	learn: 3449.1775008	total: 1.18s	remaining: 355ms
769:	learn: 3438.1914517	total: 1.18s	remaining: 353ms
770:	learn: 3427.8762753	total: 1.18s	remaining: 352ms
771:	learn: 3416.1149612	total: 1.19s	remaining: 350ms
772:	learn: 3406.3166027	total: 1.19s	remaining: 349ms
773:	learn: 3402.2142223	total: 1.19s	remaining: 347ms
774:	learn: 3388.9113959	total: 1.19s	remaining: 346ms
775:	learn: 3381.3314020	total: 1.19s	remaining: 344ms
776:	learn: 3371.6474073	total: 1.19s	remaining: 342ms
777:	learn: 3361.6339076	total: 1.19s	remaining: 341ms
778:	learn: 3354.4384144	total: 1.2s	remaining: 339ms
779:	learn: 3344.8991926	total: 1.2s	remaining: 338ms
780:	learn: 3335.1433667	total: 1.2s	remaining: 336ms
781:	learn: 3

In [196]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = -cross_val_score(pipe_4, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print("MAE scores:\n", n_scores)
# print('Accuracy: %.2f (%.2f)'% (np.mean(n_scores)))
print("Average MAE score (across experiments):")
print(n_scores.mean())

Learning rate set to 0.034961
0:	learn: 73439.4954875	total: 1.97ms	remaining: 1.97s
1:	learn: 72020.8933542	total: 3.62ms	remaining: 1.8s
2:	learn: 70680.2225818	total: 5.27ms	remaining: 1.75s
3:	learn: 69495.4921986	total: 6.97ms	remaining: 1.74s
4:	learn: 68125.5598834	total: 8.54ms	remaining: 1.7s
5:	learn: 66825.8088950	total: 10.3ms	remaining: 1.7s
6:	learn: 65789.6616592	total: 11.9ms	remaining: 1.68s
7:	learn: 64554.8482952	total: 13.4ms	remaining: 1.67s
8:	learn: 63482.2481349	total: 15ms	remaining: 1.65s
9:	learn: 62348.8341162	total: 16.6ms	remaining: 1.65s
10:	learn: 61358.3298858	total: 18.2ms	remaining: 1.63s
11:	learn: 60439.6405262	total: 19.8ms	remaining: 1.63s
12:	learn: 59495.4976191	total: 21.4ms	remaining: 1.62s
13:	learn: 58404.7101016	total: 22.9ms	remaining: 1.61s
14:	learn: 57274.4129299	total: 24.7ms	remaining: 1.62s
15:	learn: 56337.1718016	total: 26.3ms	remaining: 1.62s
16:	learn: 55392.3717463	total: 28ms	remaining: 1.62s
17:	learn: 54608.6825708	total: 29.

254:	learn: 14192.3305726	total: 385ms	remaining: 1.13s
255:	learn: 14138.6476434	total: 387ms	remaining: 1.12s
256:	learn: 14074.1333294	total: 389ms	remaining: 1.12s
257:	learn: 14032.6942081	total: 390ms	remaining: 1.12s
258:	learn: 13992.3829171	total: 392ms	remaining: 1.12s
259:	learn: 13921.2528406	total: 394ms	remaining: 1.12s
260:	learn: 13902.5705041	total: 395ms	remaining: 1.12s
261:	learn: 13884.3146132	total: 396ms	remaining: 1.12s
262:	learn: 13837.7209404	total: 398ms	remaining: 1.11s
263:	learn: 13820.6969203	total: 400ms	remaining: 1.11s
264:	learn: 13785.5033766	total: 401ms	remaining: 1.11s
265:	learn: 13738.7711651	total: 403ms	remaining: 1.11s
266:	learn: 13708.3645677	total: 405ms	remaining: 1.11s
267:	learn: 13676.2889275	total: 406ms	remaining: 1.11s
268:	learn: 13634.1145713	total: 408ms	remaining: 1.11s
269:	learn: 13555.1768203	total: 409ms	remaining: 1.11s
270:	learn: 13499.4387948	total: 411ms	remaining: 1.1s
271:	learn: 13477.5025066	total: 413ms	remaining:

524:	learn: 6395.2201481	total: 789ms	remaining: 714ms
525:	learn: 6391.7464129	total: 791ms	remaining: 713ms
526:	learn: 6355.5026365	total: 793ms	remaining: 711ms
527:	learn: 6347.0959876	total: 794ms	remaining: 710ms
528:	learn: 6327.7617062	total: 796ms	remaining: 709ms
529:	learn: 6304.6248956	total: 798ms	remaining: 707ms
530:	learn: 6293.8965024	total: 799ms	remaining: 706ms
531:	learn: 6269.3248972	total: 801ms	remaining: 705ms
532:	learn: 6259.2506367	total: 803ms	remaining: 703ms
533:	learn: 6246.3178872	total: 804ms	remaining: 702ms
534:	learn: 6219.4192850	total: 806ms	remaining: 700ms
535:	learn: 6191.3190379	total: 807ms	remaining: 699ms
536:	learn: 6172.2336992	total: 809ms	remaining: 697ms
537:	learn: 6167.1258404	total: 810ms	remaining: 696ms
538:	learn: 6141.2757609	total: 812ms	remaining: 694ms
539:	learn: 6129.7403042	total: 813ms	remaining: 693ms
540:	learn: 6102.3646804	total: 815ms	remaining: 692ms
541:	learn: 6088.7003238	total: 816ms	remaining: 690ms
542:	learn

784:	learn: 3384.3031521	total: 1.19s	remaining: 325ms
785:	learn: 3377.8271494	total: 1.19s	remaining: 323ms
786:	learn: 3374.2257264	total: 1.19s	remaining: 322ms
787:	learn: 3363.3246785	total: 1.19s	remaining: 320ms
788:	learn: 3348.0671973	total: 1.19s	remaining: 319ms
789:	learn: 3334.3265311	total: 1.19s	remaining: 317ms
790:	learn: 3323.8101495	total: 1.2s	remaining: 316ms
791:	learn: 3314.1011741	total: 1.2s	remaining: 314ms
792:	learn: 3302.5439855	total: 1.2s	remaining: 313ms
793:	learn: 3286.6964883	total: 1.2s	remaining: 311ms
794:	learn: 3274.2060534	total: 1.2s	remaining: 310ms
795:	learn: 3263.9698122	total: 1.2s	remaining: 308ms
796:	learn: 3259.5269764	total: 1.21s	remaining: 307ms
797:	learn: 3254.1214473	total: 1.21s	remaining: 306ms
798:	learn: 3244.9181455	total: 1.21s	remaining: 304ms
799:	learn: 3233.1452768	total: 1.21s	remaining: 303ms
800:	learn: 3225.2278719	total: 1.21s	remaining: 301ms
801:	learn: 3217.6553694	total: 1.21s	remaining: 300ms
802:	learn: 3210

Learning rate set to 0.034961
0:	learn: 76928.9966725	total: 1.76ms	remaining: 1.76s
1:	learn: 75452.9849085	total: 3.32ms	remaining: 1.66s
2:	learn: 73911.3089117	total: 4.9ms	remaining: 1.63s
3:	learn: 72576.6738557	total: 6.4ms	remaining: 1.59s
4:	learn: 71242.2451990	total: 7.87ms	remaining: 1.57s
5:	learn: 70019.3028925	total: 9.31ms	remaining: 1.54s
6:	learn: 68765.3876167	total: 10.8ms	remaining: 1.54s
7:	learn: 67461.2389110	total: 12.3ms	remaining: 1.53s
8:	learn: 66452.3035244	total: 13.7ms	remaining: 1.51s
9:	learn: 65390.8605182	total: 15.2ms	remaining: 1.51s
10:	learn: 64396.9886740	total: 16.9ms	remaining: 1.52s
11:	learn: 63370.2740440	total: 18.4ms	remaining: 1.52s
12:	learn: 62265.5349098	total: 19.9ms	remaining: 1.51s
13:	learn: 61350.1652329	total: 21.3ms	remaining: 1.5s
14:	learn: 60234.2327397	total: 22.9ms	remaining: 1.5s
15:	learn: 59218.3331132	total: 24.3ms	remaining: 1.49s
16:	learn: 58245.3568329	total: 25.8ms	remaining: 1.49s
17:	learn: 57280.5141966	total: 

...

937:	learn: 2032.6414696	total: 1.4s	remaining: 92.6ms
938:	learn: 2026.1812148	total: 1.4s	remaining: 91.1ms
939:	learn: 2021.2034201	total: 1.4s	remaining: 89.6ms
940:	learn: 2018.9001914	total: 1.41s	remaining: 88.1ms
941:	learn: 2014.7843938	total: 1.41s	remaining: 86.6ms
942:	learn: 2010.4032227	total: 1.41s	remaining: 85.1ms
943:	learn: 2005.8060242	total: 1.41s	remaining: 83.6ms
944:	learn: 2002.0775669	total: 1.41s	remaining: 82.2ms
945:	learn: 1998.9606934	total: 1.41s	remaining: 80.7ms
946:	learn: 1993.9132538	total: 1.41s	remaining: 79.2ms
947:	learn: 1986.1262737	total: 1.42s	remaining: 77.7ms
948:	learn: 1983.0894781	total: 1.42s	remaining: 76.2ms
949:	learn: 1975.4015518	total: 1.42s	remaining: 74.7ms
950:	learn: 1974.4724051	total: 1.42s	remaining: 73.2ms
951:	learn: 1966.8788421	total: 1.42s	remaining: 71.7ms
952:	learn: 1961.6210306	total: 1.42s	remaining: 70.2ms
953:	learn: 1954.4862664	total: 1.43s	remaining: 68.7ms
954:	learn: 1951.6138159	total: 1.43s	remaining: 67

119:	learn: 21324.9837506	total: 191ms	remaining: 1.4s
120:	learn: 21245.9902449	total: 193ms	remaining: 1.4s
121:	learn: 21121.7476075	total: 195ms	remaining: 1.4s
122:	learn: 21020.0980858	total: 196ms	remaining: 1.4s
123:	learn: 20886.5732290	total: 198ms	remaining: 1.4s
124:	learn: 20788.8076406	total: 200ms	remaining: 1.4s
125:	learn: 20684.8911207	total: 202ms	remaining: 1.4s
126:	learn: 20603.7493175	total: 203ms	remaining: 1.4s
127:	learn: 20510.0554273	total: 205ms	remaining: 1.4s
128:	learn: 20406.1373583	total: 206ms	remaining: 1.39s
129:	learn: 20300.4416546	total: 208ms	remaining: 1.39s
130:	learn: 20206.8682367	total: 210ms	remaining: 1.39s
131:	learn: 20157.2346722	total: 211ms	remaining: 1.39s
132:	learn: 20051.6515887	total: 213ms	remaining: 1.39s
133:	learn: 19975.7573020	total: 214ms	remaining: 1.39s
134:	learn: 19847.0199410	total: 216ms	remaining: 1.38s
135:	learn: 19744.0267704	total: 217ms	remaining: 1.38s
136:	learn: 19672.5024390	total: 219ms	remaining: 1.38s
1

385:	learn: 8678.3658543	total: 590ms	remaining: 938ms
386:	learn: 8650.8871352	total: 591ms	remaining: 937ms
387:	learn: 8624.1976490	total: 593ms	remaining: 935ms
388:	learn: 8586.5728509	total: 594ms	remaining: 934ms
389:	learn: 8552.7630756	total: 596ms	remaining: 932ms
390:	learn: 8531.1289677	total: 598ms	remaining: 931ms
391:	learn: 8516.9311934	total: 599ms	remaining: 930ms
392:	learn: 8499.5166331	total: 601ms	remaining: 928ms
393:	learn: 8495.1010050	total: 602ms	remaining: 926ms
394:	learn: 8460.3412200	total: 604ms	remaining: 925ms
395:	learn: 8421.6390055	total: 606ms	remaining: 924ms
396:	learn: 8415.0566908	total: 607ms	remaining: 922ms
397:	learn: 8387.4710018	total: 609ms	remaining: 920ms
398:	learn: 8370.3072004	total: 610ms	remaining: 919ms
399:	learn: 8358.3289241	total: 611ms	remaining: 917ms
400:	learn: 8323.8635319	total: 613ms	remaining: 915ms
401:	learn: 8318.2830028	total: 614ms	remaining: 914ms
402:	learn: 8287.2032891	total: 616ms	remaining: 913ms
403:	learn

656:	learn: 4596.9110831	total: 988ms	remaining: 516ms
657:	learn: 4578.0995772	total: 990ms	remaining: 514ms
658:	learn: 4564.6624470	total: 991ms	remaining: 513ms
659:	learn: 4546.0009292	total: 993ms	remaining: 511ms
660:	learn: 4536.6778326	total: 994ms	remaining: 510ms
661:	learn: 4527.5195492	total: 996ms	remaining: 508ms
662:	learn: 4516.7949864	total: 998ms	remaining: 507ms
663:	learn: 4503.8047579	total: 999ms	remaining: 506ms
664:	learn: 4496.0289767	total: 1s	remaining: 504ms
665:	learn: 4480.3904450	total: 1s	remaining: 503ms
666:	learn: 4471.8046032	total: 1s	remaining: 501ms
667:	learn: 4461.5279375	total: 1s	remaining: 500ms
668:	learn: 4452.3086757	total: 1.01s	remaining: 498ms
669:	learn: 4437.1229299	total: 1.01s	remaining: 497ms
670:	learn: 4419.9020726	total: 1.01s	remaining: 495ms
671:	learn: 4418.5990553	total: 1.01s	remaining: 494ms
672:	learn: 4408.7423721	total: 1.01s	remaining: 492ms
673:	learn: 4392.7942315	total: 1.01s	remaining: 491ms
674:	learn: 4384.21200

916:	learn: 2346.0157166	total: 1.38s	remaining: 125ms
917:	learn: 2342.0723087	total: 1.38s	remaining: 123ms
918:	learn: 2333.6131455	total: 1.38s	remaining: 122ms
919:	learn: 2325.7809935	total: 1.38s	remaining: 120ms
920:	learn: 2325.4198625	total: 1.39s	remaining: 119ms
921:	learn: 2322.2303614	total: 1.39s	remaining: 117ms
922:	learn: 2317.5132826	total: 1.39s	remaining: 116ms
923:	learn: 2309.3091376	total: 1.39s	remaining: 114ms
924:	learn: 2303.7140373	total: 1.39s	remaining: 113ms
925:	learn: 2297.5712338	total: 1.39s	remaining: 111ms
926:	learn: 2291.8135113	total: 1.4s	remaining: 110ms
927:	learn: 2283.1784754	total: 1.4s	remaining: 108ms
928:	learn: 2274.0347860	total: 1.4s	remaining: 107ms
929:	learn: 2269.4822584	total: 1.4s	remaining: 105ms
930:	learn: 2260.9276098	total: 1.4s	remaining: 104ms
931:	learn: 2256.1720106	total: 1.4s	remaining: 102ms
932:	learn: 2247.0223486	total: 1.4s	remaining: 101ms
933:	learn: 2236.5711344	total: 1.41s	remaining: 99.3ms
934:	learn: 2232

 ### <a id="V.5"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.5 CatBoostRegressor</p>

In [197]:
pipe_5 = make_pipeline(preprocessor, selection,  CatBoostRegressor(iterations=3000, 
    depth=6, 
    learning_rate=0.03,
    loss_function='RMSE',
    verbose=200, random_state=42))

In [198]:
pipe_5.fit(X,y)

0:	learn: 75877.5097257	total: 1.98ms	remaining: 5.93s
200:	learn: 17171.2423533	total: 331ms	remaining: 4.61s
400:	learn: 10462.7738450	total: 642ms	remaining: 4.16s
600:	learn: 7044.0813737	total: 955ms	remaining: 3.81s
800:	learn: 4807.3662264	total: 1.27s	remaining: 3.5s
1000:	learn: 3325.6767573	total: 1.6s	remaining: 3.2s
1200:	learn: 2419.4693195	total: 1.93s	remaining: 2.89s
1400:	learn: 1683.1660164	total: 2.26s	remaining: 2.58s
1600:	learn: 1270.7613389	total: 2.59s	remaining: 2.26s
1800:	learn: 953.0733962	total: 2.93s	remaining: 1.95s
2000:	learn: 707.7314231	total: 3.25s	remaining: 1.62s
2200:	learn: 526.2095464	total: 3.58s	remaining: 1.3s
2400:	learn: 391.7870891	total: 3.9s	remaining: 974ms
2600:	learn: 294.6292806	total: 4.22s	remaining: 648ms
2800:	learn: 222.2395019	total: 4.54s	remaining: 323ms
2999:	learn: 170.8155844	total: 4.86s	remaining: 0us


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('powertransformer',
                                                                   PowerTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D0107990>),
                                                 ('pipeline-2',
                                                  Pip...
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001A1D017B6D0>)])),
                ('selectfrommodel',
                 SelectFromModel(estimator=Lasso(alpha=0.3, random_state=42))),
                ('catboostregressor',
                 <catboost.core.CatBoostRegressor object at 0x000001A1D4F94E10>)])

 ### <a id="V.5.1"><p style="background-color: #294B8E;color:white;display: inline-block;padding:.6rem;border-radius:.5rem">V.5.1 CatBoostRegressor - evaluation</p>

In [199]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = cross_val_score(pipe_5, X, y, cv=kf, scoring='r2')
print('R2_score: %.2f (%.2f)'% (np.mean(n_scores), np.std(n_scores)))

0:	learn: 73633.5382034	total: 2.01ms	remaining: 6.04s
200:	learn: 18824.9257459	total: 314ms	remaining: 4.38s
400:	learn: 11041.3258541	total: 604ms	remaining: 3.92s
600:	learn: 6625.4268129	total: 899ms	remaining: 3.59s
800:	learn: 4247.2868963	total: 1.2s	remaining: 3.29s
1000:	learn: 2722.1855159	total: 1.51s	remaining: 3.01s
1200:	learn: 1808.5804360	total: 1.8s	remaining: 2.7s
1400:	learn: 1213.3135282	total: 2.1s	remaining: 2.4s
1600:	learn: 842.2513066	total: 2.4s	remaining: 2.1s
1800:	learn: 563.1995165	total: 2.72s	remaining: 1.81s
2000:	learn: 386.1247064	total: 3.02s	remaining: 1.51s
2200:	learn: 276.7946026	total: 3.33s	remaining: 1.21s
2400:	learn: 197.7673314	total: 3.63s	remaining: 905ms
2600:	learn: 144.7583390	total: 3.92s	remaining: 602ms
2800:	learn: 106.0282108	total: 4.22s	remaining: 300ms
2999:	learn: 77.3751656	total: 4.52s	remaining: 0us
0:	learn: 77125.6957105	total: 1.93ms	remaining: 5.79s
200:	learn: 17130.1634224	total: 299ms	remaining: 4.17s
400:	learn: 10

In [200]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
# cross_val_score(pipe, X, y, cv=kf, scoring='r2').mean()

n_scores = -cross_val_score(pipe_4, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print("MAE scores:\n", n_scores)
# print('Accuracy: %.2f (%.2f)'% (np.mean(n_scores)))
print("Average MAE score (across experiments):")
print(n_scores.mean())

Learning rate set to 0.034961
0:	learn: 73439.4954875	total: 1.97ms	remaining: 1.97s
1:	learn: 72020.8933542	total: 3.83ms	remaining: 1.91s
2:	learn: 70680.2225818	total: 5.5ms	remaining: 1.83s
3:	learn: 69495.4921986	total: 7.12ms	remaining: 1.77s
4:	learn: 68125.5598834	total: 8.88ms	remaining: 1.77s
5:	learn: 66825.8088950	total: 10.6ms	remaining: 1.76s
6:	learn: 65789.6616592	total: 12.3ms	remaining: 1.75s
7:	learn: 64554.8482952	total: 14.2ms	remaining: 1.76s
8:	learn: 63482.2481349	total: 15.9ms	remaining: 1.75s
9:	learn: 62348.8341162	total: 17.5ms	remaining: 1.73s
10:	learn: 61358.3298858	total: 19.2ms	remaining: 1.72s
11:	learn: 60439.6405262	total: 20.8ms	remaining: 1.71s
12:	learn: 59495.4976191	total: 22.5ms	remaining: 1.71s
13:	learn: 58404.7101016	total: 24.3ms	remaining: 1.71s
14:	learn: 57274.4129299	total: 25.9ms	remaining: 1.7s
15:	learn: 56337.1718016	total: 27.6ms	remaining: 1.7s
16:	learn: 55392.3717463	total: 29.1ms	remaining: 1.68s
17:	learn: 54608.6825708	total:

249:	learn: 14522.1171305	total: 388ms	remaining: 1.16s
250:	learn: 14425.5451927	total: 389ms	remaining: 1.16s
251:	learn: 14326.2842647	total: 391ms	remaining: 1.16s
252:	learn: 14289.5603349	total: 393ms	remaining: 1.16s
253:	learn: 14248.0574694	total: 394ms	remaining: 1.16s
254:	learn: 14192.3305726	total: 396ms	remaining: 1.16s
255:	learn: 14138.6476434	total: 398ms	remaining: 1.16s
256:	learn: 14074.1333294	total: 399ms	remaining: 1.15s
257:	learn: 14032.6942081	total: 401ms	remaining: 1.15s
258:	learn: 13992.3829171	total: 403ms	remaining: 1.15s
259:	learn: 13921.2528406	total: 404ms	remaining: 1.15s
260:	learn: 13902.5705041	total: 406ms	remaining: 1.15s
261:	learn: 13884.3146132	total: 407ms	remaining: 1.15s
262:	learn: 13837.7209404	total: 409ms	remaining: 1.15s
263:	learn: 13820.6969203	total: 410ms	remaining: 1.14s
264:	learn: 13785.5033766	total: 412ms	remaining: 1.14s
265:	learn: 13738.7711651	total: 413ms	remaining: 1.14s
266:	learn: 13708.3645677	total: 415ms	remaining

514:	learn: 6566.9109621	total: 789ms	remaining: 743ms
515:	learn: 6559.3094110	total: 790ms	remaining: 741ms
516:	learn: 6539.0102311	total: 792ms	remaining: 740ms
517:	learn: 6523.8668326	total: 794ms	remaining: 739ms
518:	learn: 6514.6474045	total: 795ms	remaining: 737ms
519:	learn: 6497.4846362	total: 797ms	remaining: 736ms
520:	learn: 6495.4111161	total: 799ms	remaining: 734ms
521:	learn: 6469.2171317	total: 801ms	remaining: 733ms
522:	learn: 6438.7332405	total: 802ms	remaining: 731ms
523:	learn: 6415.7869652	total: 804ms	remaining: 730ms
524:	learn: 6395.2201481	total: 805ms	remaining: 728ms
525:	learn: 6391.7464129	total: 807ms	remaining: 727ms
526:	learn: 6355.5026365	total: 808ms	remaining: 725ms
527:	learn: 6347.0959876	total: 810ms	remaining: 724ms
528:	learn: 6327.7617062	total: 811ms	remaining: 722ms
529:	learn: 6304.6248956	total: 813ms	remaining: 721ms
530:	learn: 6293.8965024	total: 814ms	remaining: 719ms
531:	learn: 6269.3248972	total: 816ms	remaining: 718ms
532:	learn

763:	learn: 3579.4497287	total: 1.17s	remaining: 363ms
764:	learn: 3574.0272017	total: 1.18s	remaining: 361ms
765:	learn: 3559.8662852	total: 1.18s	remaining: 360ms
766:	learn: 3546.5841835	total: 1.18s	remaining: 358ms
767:	learn: 3543.9599656	total: 1.18s	remaining: 357ms
768:	learn: 3530.8020374	total: 1.18s	remaining: 355ms
769:	learn: 3520.4288783	total: 1.18s	remaining: 354ms
770:	learn: 3517.7196549	total: 1.19s	remaining: 352ms
771:	learn: 3510.0115597	total: 1.19s	remaining: 351ms
772:	learn: 3498.2061253	total: 1.19s	remaining: 349ms
773:	learn: 3485.3816868	total: 1.19s	remaining: 348ms
774:	learn: 3478.9216991	total: 1.19s	remaining: 346ms
775:	learn: 3474.8578979	total: 1.19s	remaining: 345ms
776:	learn: 3457.3443430	total: 1.2s	remaining: 343ms
777:	learn: 3449.5119608	total: 1.2s	remaining: 342ms
778:	learn: 3442.6319519	total: 1.2s	remaining: 340ms
779:	learn: 3431.0267896	total: 1.2s	remaining: 339ms
780:	learn: 3420.9688733	total: 1.2s	remaining: 337ms
781:	learn: 341

Learning rate set to 0.034961
0:	learn: 76928.9966725	total: 1.95ms	remaining: 1.95s
1:	learn: 75452.9849085	total: 3.64ms	remaining: 1.82s
2:	learn: 73911.3089117	total: 5.13ms	remaining: 1.7s
3:	learn: 72576.6738557	total: 6.6ms	remaining: 1.64s
4:	learn: 71242.2451990	total: 8.11ms	remaining: 1.61s
5:	learn: 70019.3028925	total: 9.53ms	remaining: 1.58s
6:	learn: 68765.3876167	total: 11ms	remaining: 1.56s
7:	learn: 67461.2389110	total: 12.4ms	remaining: 1.54s
8:	learn: 66452.3035244	total: 13.9ms	remaining: 1.53s
9:	learn: 65390.8605182	total: 15.4ms	remaining: 1.52s
10:	learn: 64396.9886740	total: 16.9ms	remaining: 1.52s
11:	learn: 63370.2740440	total: 18.4ms	remaining: 1.51s
12:	learn: 62265.5349098	total: 19.8ms	remaining: 1.5s
13:	learn: 61350.1652329	total: 21.3ms	remaining: 1.5s
14:	learn: 60234.2327397	total: 22.8ms	remaining: 1.5s
15:	learn: 59218.3331132	total: 24.3ms	remaining: 1.49s
16:	learn: 58245.3568329	total: 25.7ms	remaining: 1.49s
17:	learn: 57280.5141966	total: 27.

...

816:	learn: 2681.5071928	total: 1.2s	remaining: 269ms
817:	learn: 2678.4909017	total: 1.2s	remaining: 267ms
818:	learn: 2670.8422820	total: 1.2s	remaining: 266ms
819:	learn: 2660.4863657	total: 1.2s	remaining: 264ms
820:	learn: 2648.7667026	total: 1.21s	remaining: 263ms
821:	learn: 2642.8963146	total: 1.21s	remaining: 261ms
822:	learn: 2638.1657410	total: 1.21s	remaining: 260ms
823:	learn: 2629.7934243	total: 1.21s	remaining: 259ms
824:	learn: 2627.5774290	total: 1.21s	remaining: 257ms
825:	learn: 2622.6541324	total: 1.21s	remaining: 256ms
826:	learn: 2613.7146160	total: 1.22s	remaining: 254ms
827:	learn: 2606.8785056	total: 1.22s	remaining: 253ms
828:	learn: 2597.8121018	total: 1.22s	remaining: 251ms
829:	learn: 2587.5835601	total: 1.22s	remaining: 250ms
830:	learn: 2581.6020438	total: 1.22s	remaining: 248ms
831:	learn: 2571.1511465	total: 1.22s	remaining: 247ms
832:	learn: 2565.6890582	total: 1.22s	remaining: 245ms
833:	learn: 2559.3553461	total: 1.23s	remaining: 244ms
834:	learn: 25

Learning rate set to 0.034961
0:	learn: 76731.3955966	total: 2.11ms	remaining: 2.11s
1:	learn: 75256.6323407	total: 4.06ms	remaining: 2.02s
2:	learn: 74113.6122058	total: 5.94ms	remaining: 1.97s
3:	learn: 72882.3655416	total: 7.85ms	remaining: 1.95s
4:	learn: 71438.6667487	total: 9.63ms	remaining: 1.92s
5:	learn: 70227.1307089	total: 11.5ms	remaining: 1.9s
6:	learn: 69078.0583396	total: 13.4ms	remaining: 1.9s
7:	learn: 67795.0959557	total: 15.3ms	remaining: 1.9s
8:	learn: 66598.6485833	total: 17.2ms	remaining: 1.9s
9:	learn: 65461.9938538	total: 19.2ms	remaining: 1.9s
10:	learn: 64169.2888446	total: 21.2ms	remaining: 1.91s
11:	learn: 63016.3719314	total: 23ms	remaining: 1.89s
12:	learn: 61980.9879210	total: 24.9ms	remaining: 1.89s
13:	learn: 60960.5615893	total: 26.7ms	remaining: 1.88s
14:	learn: 59826.8842013	total: 28.5ms	remaining: 1.87s
15:	learn: 58746.5189223	total: 30.2ms	remaining: 1.86s
16:	learn: 57745.8080528	total: 32.2ms	remaining: 1.86s
17:	learn: 56671.2900779	total: 34m

255:	learn: 13186.4790642	total: 406ms	remaining: 1.18s
256:	learn: 13177.2024899	total: 408ms	remaining: 1.18s
257:	learn: 13115.2052510	total: 410ms	remaining: 1.18s
258:	learn: 13098.5677394	total: 411ms	remaining: 1.18s
259:	learn: 13066.1447314	total: 413ms	remaining: 1.17s
260:	learn: 13055.0274959	total: 414ms	remaining: 1.17s
261:	learn: 12991.9805182	total: 416ms	remaining: 1.17s
262:	learn: 12932.0874430	total: 418ms	remaining: 1.17s
263:	learn: 12880.8070760	total: 419ms	remaining: 1.17s
264:	learn: 12848.4489279	total: 421ms	remaining: 1.17s
265:	learn: 12825.4351351	total: 423ms	remaining: 1.17s
266:	learn: 12799.3692836	total: 424ms	remaining: 1.16s
267:	learn: 12744.7938582	total: 426ms	remaining: 1.16s
268:	learn: 12718.9828795	total: 428ms	remaining: 1.16s
269:	learn: 12665.8102634	total: 429ms	remaining: 1.16s
270:	learn: 12592.4943199	total: 431ms	remaining: 1.16s
271:	learn: 12536.3683338	total: 432ms	remaining: 1.16s
272:	learn: 12473.1121012	total: 434ms	remaining

505:	learn: 6499.9447101	total: 803ms	remaining: 784ms
506:	learn: 6492.8877047	total: 805ms	remaining: 783ms
507:	learn: 6466.5407708	total: 807ms	remaining: 782ms
508:	learn: 6446.1818686	total: 809ms	remaining: 780ms
509:	learn: 6421.1670920	total: 810ms	remaining: 779ms
510:	learn: 6419.1844162	total: 812ms	remaining: 777ms
511:	learn: 6405.7474551	total: 814ms	remaining: 776ms
512:	learn: 6392.9347266	total: 815ms	remaining: 774ms
513:	learn: 6367.1456992	total: 817ms	remaining: 773ms
514:	learn: 6347.0172148	total: 819ms	remaining: 771ms
515:	learn: 6344.2314518	total: 821ms	remaining: 770ms
516:	learn: 6327.5050041	total: 822ms	remaining: 768ms
517:	learn: 6314.0122543	total: 824ms	remaining: 767ms
518:	learn: 6281.5904175	total: 826ms	remaining: 765ms
519:	learn: 6251.0736134	total: 827ms	remaining: 763ms
520:	learn: 6224.4479393	total: 829ms	remaining: 762ms
521:	learn: 6204.8604840	total: 830ms	remaining: 760ms
522:	learn: 6191.2703016	total: 832ms	remaining: 759ms
523:	learn

762:	learn: 3496.2249203	total: 1.21s	remaining: 375ms
763:	learn: 3490.9696482	total: 1.21s	remaining: 373ms
764:	learn: 3482.0673178	total: 1.21s	remaining: 372ms
765:	learn: 3475.3184189	total: 1.21s	remaining: 370ms
766:	learn: 3463.1659940	total: 1.21s	remaining: 369ms
767:	learn: 3455.9224899	total: 1.21s	remaining: 367ms
768:	learn: 3449.1775008	total: 1.22s	remaining: 365ms
769:	learn: 3438.1914517	total: 1.22s	remaining: 364ms
770:	learn: 3427.8762753	total: 1.22s	remaining: 362ms
771:	learn: 3416.1149612	total: 1.22s	remaining: 361ms
772:	learn: 3406.3166027	total: 1.22s	remaining: 359ms
773:	learn: 3402.2142223	total: 1.22s	remaining: 358ms
774:	learn: 3388.9113959	total: 1.23s	remaining: 356ms
775:	learn: 3381.3314020	total: 1.23s	remaining: 354ms
776:	learn: 3371.6474073	total: 1.23s	remaining: 353ms
777:	learn: 3361.6339076	total: 1.23s	remaining: 351ms
778:	learn: 3354.4384144	total: 1.23s	remaining: 350ms
779:	learn: 3344.8991926	total: 1.23s	remaining: 348ms
780:	learn

<a id="VI"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700">VI <span style='color:#CDA63A'>|</span>CONCLUSION 👩‍🏫</div>